In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('Reviews.csv')

In [3]:
data_score_removed = df[df['Score']!=3]  
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

score_upd = data_score_removed['Score']
t = score_upd.map(partition)
data_score_removed['Score']=t

<ipython-input-3-8068903bead5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_score_removed['Score']=t


In [4]:
final_data = data_score_removed.drop_duplicates(subset={"UserId","ProfileName","Time","Text"})
final = final_data[final_data['HelpfulnessNumerator'] <= final_data['HelpfulnessDenominator']]

In [5]:
final=final.dropna()
final.reset_index(inplace=True)

In [6]:
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
corpus_lemma = []
for i in range(0, 50000):
    review = re.sub('[^a-zA-Z]', ' ', final['Summary'][i]+final['Text'][i])
    review = review.lower()
    review = review.split()
    
    review = [str(lemmatizer.lemmatize(word)) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus_lemma.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amard\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amard\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=35000,ngram_range=(1,3))
X=tfidf_v.fit_transform(corpus_lemma).toarray()

In [ ]:
X.shape
y = final['Score'][:50000]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [ ]:
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Reds):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else: 
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier 
linear_clf =  PassiveAggressiveClassifier(C = 0.5, random_state = 5) 
from sklearn import metrics
import numpy as np
import itertools

In [ ]:
linear_clf.fit(X_train, y_train)
pred = linear_clf.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
cm = metrics.confusion_matrix(y_test, pred)
plot_confusion_matrix(cm, classes=['FAKE Data', 'REAL Data'])

In [ ]:
X_train = corpus_lemma
Y_train = final['Score'][:50000]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=15000,ngram_range=(1,3))),
    ('clf', linear_model.PassiveAggressiveClassifier(C = 0.5, random_state = 5) ),
])
pipeline.fit(X_train, Y_train)

In [ ]:
pipeline.predict(["bad taste"])

In [ ]:
filename = 'pipeline.sav'
joblib.dump(pipeline, filename)
filename = './pipeline.sav'

In [ ]:
loaded_model = joblib.load(filename)
result = loaded_model.predict(["taste great"])
print(result) 